In [1]:
from tqdm import tqdm
from collections import OrderedDict
import logging
import logging.config

from pymongo import MongoClient
import numpy as np

import annonutils
import common
import db_to_aids as db2a
from _log_ import logcfg

log = logging.getLogger(__name__)
logging.config.dictConfig(logcfg)

In [2]:
datacfg = db2a.dbcfg
cfg = db2a.appcfg
get_annon_data = db2a.get_annon_data
do_aids_split = db2a.do_aids_split

log.info("datacfg:{}".format(datacfg))

2019-10-08 10:46:53,864:[INFO]:[__main__]:[<ipython-input-2-50d89f1a0cde>:6 -             <module>() ]: datacfg:{'created_on': None, 'modified_on': None, 'timestamp': None, 'anndb_id': None, 'rel_id': None, 'dbname': None, 'dbid': None, 'allowed_file_type': ['.txt', '.csv', '.yml', '.json'], 'allowed_image_type': ['.pdf', '.png', '.jpg', '.jpeg', '.gif'], 'allowed_video_type': ['.mp4'], 'dataset': {}, 'load_data_from_file': False, 'train': [], 'evaluate': [], 'predict': [], 'publish': [], 'report': [], 'description': 'HD Map Dataset', 'files': {}, 'id': 'hmd', 'name': 'hmd', 'problem_id': 'hmd', 'annon_type': 'hmd', 'dataclass': 'HmdDataset', 'classes': '', 'class_ids': None, 'class_map': None, 'num_classes': None, 'splits': None, 'stats': {}, 'data_read_threshold': -1, 'db_dir': None, 'return_hmd': None, 'train_mode': 'training', 'test_mode': 'inference'}


In [3]:
DBCFG = cfg['DBCFG']
mclient = MongoClient('mongodb://'+DBCFG['HOST']+':'+str(DBCFG['PORT']))
db = mclient[DBCFG['DBNAME']]

In [4]:
## keep the master list of row indices (image) which has been processed
already_processed_indices = np.array([], dtype='int32')
splited_indices = {}

## percentage split between train and val data; remaining is the test data
prcntg = np.array([.7,.2])


In [9]:
## Get Images and Classinfo (categories/labels)
tblname = annonutils.get_tblname('IMAGES')
collection = db.get_collection(tblname)

images = np.array(list(collection.find({}, {'_id':False})))
log.info("len(images): {}".format(len(images)))
# log.info(images)

tblname = annonutils.get_tblname('CLASSINFO')
collection = db.get_collection(tblname)
cur = list(collection.find({}, {'_id':False}))
lbl_ids = []
for item in cur:
    lbl_ids.append(item['lbl_id'])

log.info('len(lbl_ids): {}'.format(len(lbl_ids[1:])))
log.info('lbl_ids: {}'.format(lbl_ids[1:]))

lbl_ids[1:] = np.sort(lbl_ids[1:])
log

2019-10-08 10:52:04,644:[INFO]:[__main__]:[<ipython-input-9-f4fc9e2ff579>:6 -             <module>() ]: len(images): 30
2019-10-08 10:52:04,646:[INFO]:[__main__]:[<ipython-input-9-f4fc9e2ff579>:16 -             <module>() ]: len(lbl_ids): 6
2019-10-08 10:52:04,646:[INFO]:[__main__]:[<ipython-input-9-f4fc9e2ff579>:17 -             <module>() ]: lbl_ids: ['lane_marking', 'signage', 'road_polygon', 'loose_material', 'traffic_sign', 'traffic_sign_frame']


In [47]:
## filter specific labels
AIDS_FILTER = {
    'LABELS':[]
    ,'RATIO':0
    ,'BY':['annotation_per_label','image_per_label']
    ,'ON':'annotation_per_label'
}
tblname = annonutils.get_tblname('IMAGES')
collection = db.get_collection(tblname)

labels = ["signage"]
labels = ["footpath_polygon"]
query = {}
query = {"lbl_ids": { "$in": labels }}

images = np.array(list(collection.find(query, {'_id':False})))

log.info("len(images): {}".format(len(images)))
# log.info("images: {}".format(images))

lbl_ids = images[0]['lbl_ids']
lbl_ids.index(labels[0])
print(filter(lambda x: labels in x, images[0]['lbl_ids']))

[lbl_ids.]
# images['annotations']

2019-09-27 23:52:45,808:[INFO]:[__main__]:[<ipython-input-47-1b94b120dee7>:18 -             <module>() ]: len(images): 85


[0]

In [6]:
## https://stackoverflow.com/questions/17506163/how-to-convert-a-boolean-array-to-an-int-array
## https://stackoverflow.com/questions/20373039/how-do-i-convert-a-numpy-matrix-into-a-boolean-matrix

tensor = np.zeros([len(images), len(lbl_ids[1:])], int)
lbls = OrderedDict({j:0 for i, j in enumerate(lbl_ids[1:])})
for j, image in enumerate(images):
    labels_all = lbls.copy()
    for l in image['lbl_ids']:
        labels_all[l] += 1
    label_cols = list(labels_all.values())
    tensor[j,:] = label_cols

# print("{}".format(tensor))
# print(np.array(tensor, dtype=bool))

labels = list(lbls.keys())
print("{}".format(labels))

['footpath_polygon', 'barricade', 'signage', 'traffic_sign', 'traffic_light']


In [7]:
def do_db_split(tensor, labels, splited_indices, already_processed_indices, prcntg):
    print("--------------------------------------->")
    print("tensor.shape: {}\nlabels to be processed: {}\nalready_processed_indices: {}\nprcntg: {}".format(tensor.shape, labels, already_processed_indices, prcntg))

    dd = np.array(tensor != 0, dtype=int)
    total_images_per_label, total_label_per_image = np.sum(dd, axis=0), np.sum(dd, axis=1)
    total_duplications_in_image = np.sum(np.sum(dd, axis=1)) - len(np.sum(dd, axis=1))
    total_labels, total_annotations_per_label = len(np.sum(tensor, axis=0)), np.sum(tensor, axis=0)
    total_annotations = np.sum(np.sum(tensor, axis=0))
    total_images, total_annotations_per_image = len(np.sum(tensor, axis=1)), np.sum(tensor, axis=1)

    print("Total Images per Label: {}\nTotal Label per Image: {}".format(total_images_per_label, total_label_per_image))
    print("Total Duplication in Images: {}".format(total_duplications_in_image))
    print("Total Labels: {}\nTotal Annotations per Label: {}".format(total_labels, total_annotations_per_label))
    print("Total Annotations: {}".format(total_annotations))
    print("Total Images: {}\nTotal Annotations per Image: {}".format(total_images, total_annotations_per_image))
    
    if total_images > 0 and len(labels) > 0:
        print("total_images_per_label: {}".format(total_images_per_label))
        total_images_per_label_sorted_index = np.argsort(total_images_per_label)
        print("total_images_per_label_sorted_index: {}".format(total_images_per_label_sorted_index))

        ## index_of_min_total_images_per_label
        index = total_images_per_label_sorted_index[0]
        print("total_images_per_label[index]: {}".format(total_images_per_label[index]))

        ## column values for the given index_of_min_total_images_per_label
        index_col_vals = tensor[:,index]
        print("index_col_vals: {}".format(index_col_vals))

        label = labels[index]
        print("label: {}".format(label))

        ## for the given index_col_vals get only the non-zero indices in an array i.e. only the rows (i.e. images) which has the labels
        K = np.where(index_col_vals != 0)[0]
        print("len(K): {}, K: {}".format(len(K), K))

        ## calculate the total_images_per_label for the selected column i.e. label
        T = np.sum(index_col_vals != 0)
        print("total_images_per_label_col(T): {}".format(T))

        ## must verfiy the total length calculate in different ways must match
        assert T == len(K)

        ## Check and filter rows(images) which are aleady processed
        print("len(already_processed_indices), already_processed_indices: {}".format(len(already_processed_indices), already_processed_indices))
        
        if len(already_processed_indices) > 0:
            K = K[np.where(np.in1d(K,already_processed_indices) < 1 )[0]]
            print("K: {}".format(K))
            T = len(K)
        
        if len(K) > 0:
            ## Get the split distribution based on the 'T'
            ptn, pvl = prcntg[0],prcntg[1]
            tn,vl = int(T*ptn), int(T*pvl)
            tt = T - tn - vl
            print("train, val, test splits: {},{},{}".format(tn, vl, tt))

            assert T == tn+vl+tt
            print("tn, tn+vl, tt: {},{},{}".format(tn, tn+vl, tt))

            ## get the indices per dataset splits
            split_pts = [[0,tn-1], [tn,tn+vl-1], [tn+vl,T-1]]
            print("split_pts: {}".format(split_pts))

            print("K[0:tn], K[tn:tn+vl], K[tn+vl:T]: {},{},{}".format(K[0:tn], K[tn:tn+vl], K[tn+vl:T]))
            
            if label not in splited_indices:
                splited_indices[label] = {}
            else:
                print("label already exists!!! raise Error")

#             splited_indices[label]["splits"] = np.array([K[0:tn], K[tn:tn+vl], K[tn+vl:T]])
            splited_indices[label]["splits"] = [ list(K[0:tn]), list(K[tn:tn+vl]), list(K[tn+vl:T]) ]
            splited_indices[label]["total_images_per_label"] = total_images_per_label[0]
            splited_indices[label]["label"] = label
            
            ## split the indices for the rows i.e. the images which has the labels in the given split percentage
            already_processed_indices = np.concatenate((already_processed_indices, K), axis=0)
            print("len(already_processed_indices), already_processed_indices: {}".format(len(already_processed_indices), already_processed_indices))
            
            print("splited_indices: {}".format(splited_indices))
            if len(labels) > 0:
                print("index: {}, labels: {}".format(index, labels))
                labels.pop(index)
                print("labels: {}".format(labels))
                ## rows for a particular column which has value, make them as zero for all the rows i.e. for the images
                tensor[:,index][K[0:tn]] = np.zeros(len(tensor[:,index][K[0:tn]]), dtype=int)
                tensor[:,index][K[tn:tn+vl]] = np.zeros(len(tensor[:,index][K[tn:tn+vl]]), dtype=int)
                tensor[:,index][K[tn+vl:T]] = np.zeros(len(tensor[:,index][K[tn+vl:T]]), dtype=int)

                ## CAUTIOUS: recurssion!!!
                tensor = tensor[:,total_images_per_label_sorted_index[1:]]
                do_db_split(tensor, labels, splited_indices, already_processed_indices, prcntg)



In [8]:
do_db_split(tensor, labels, splited_indices, already_processed_indices, prcntg)

--------------------------------------->
tensor.shape: (98, 5)
labels to be processed: ['footpath_polygon', 'barricade', 'signage', 'traffic_sign', 'traffic_light']
already_processed_indices: []
prcntg: [0.7 0.2]
Total Images per Label: [85 11 17 18 16]
Total Label per Image: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 2 2 2 2 1 1 1 1 2 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 3 1 3 2 1 3 2 1 3 2 1
 3 2 1 3 3 1 3 2 1 4 1 1 4 2 1 4 2 1 3 2 2 3 2 1]
Total Duplication in Images: 49
Total Labels: 5
Total Annotations per Label: [106  11  17  18  16]
Total Annotations: 168
Total Images: 98
Total Annotations per Image: [1 2 2 1 1 1 1 2 2 1 1 2 2 2 2 1 1 2 2 3 3 1 1 3 3 2 2 1 1 1 1 3 3 1 1 1 1
 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 1 4 2 1 3 2 1 3 2 1
 3 2 1 3 3 1 3 2 1 4 1 1 4 2 1 4 2 1 3 2 2 3 2 1]
total_images_per_label: [85 11 17 18 16]
total_images_per_label_sorted_index: [1 4 2 3 0]
total_images_per_label[index]: 11
index_col_vals: [0 0 0 0 0 0 0 

In [10]:
print("splited_indices: {}".format(splited_indices))

splited_indices: {'barricade': {'splits': [[19, 20, 23, 24, 25, 26, 31], [32, 87], [92, 95]], 'total_images_per_label': 85, 'label': 'barricade'}, 'footpath_polygon': {'splits': [[82, 83, 84, 85, 86, 88, 89, 90, 91], [93, 94], [96, 97]], 'total_images_per_label': 16, 'label': 'footpath_polygon'}, 'signage': {'splits': [[61, 62, 63, 65, 66, 68, 69, 71, 72], [74, 75], [77, 78, 80]], 'total_images_per_label': 17, 'label': 'signage'}, 'traffic_sign': {'splits': [[70, 73], [], [81]], 'total_images_per_label': 18, 'label': 'traffic_sign'}, 'traffic_light': {'splits': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46], [47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57], [58, 59, 60, 64, 67, 76, 79]], 'total_images_per_label': 85, 'label': 'traffic_light'}}


In [13]:
images[ [19, 20, 23, 24, 25, 26, 31] ]

array([{'img_id': 'img-dbf4a2c4-374d-4eeb-bed3-4f1d3189b11b', 'annon_dir': '240719_215430/annon_data', 'annotations': ['ant-d4678091-8b23-4bfa-a0d2-807467094716', 'ant-88fafd3e-d04c-42be-a2de-5b3c9070a3db', 'ant-1ce7b6db-dd99-41cf-871f-8f9bf20d9a6f'], 'base_dir': 'images-p1-070619_AT5', 'created_on': '2019-07-24 21:54:32 +05:30', 'dir': 'images/images-p1-070619_AT5', 'file_attributes': {'img_id': 'img-dbf4a2c4-374d-4eeb-bed3-4f1d3189b11b', 'footpath_polygon': 2, 'barricade': 1}, 'file_id': '271218_120802_16717_zed_l_091.jpg1457700', 'filename': '271218_120802_16717_zed_l_091.jpg', 'filepath': '/aimldl-dat/data-gaze/AIML_Annotation/ods_merged_on_050719/images/images-p1-070619_AT5/271218_120802_16717_zed_l_091.jpg', 'height': 1080, 'lbl_ids': ['footpath_polygon', 'footpath_polygon', 'barricade'], 'modified_on': None, 'rel_filename': 'images-p1-070619_AT5_via205_110619.json', 'size': 1457700, 'width': 1920},
       {'img_id': 'img-2e7e6acf-ed6e-49cc-b010-d442905423e0', 'annon_dir': '24071